<a href="https://colab.research.google.com/github/Ashvitharavichandran/Marketing_Analysis_Project/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#library
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
#Load Dataset
df = pd.read_csv("Groceries_dataset.csv")
print(df.shape)

(38765, 3)


In [ ]:
df.head(5)

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [ ]:
df.isnull().sum().sort_values(ascending= False)

,0
Member_number,0
Date,0
itemDescription,0


In [ ]:
# Data Cleaning
# Convert date
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df["Date"]


,Date
0,2015-07-21
1,2015-01-05
2,2015-09-19
3,2015-12-12
4,2015-02-01
...,...
38760,2014-10-08
38761,2014-02-23
38762,2014-04-16
38763,2014-12-03


In [ ]:
# Create Transaction ID
df["Transaction_ID"] = (
    df["Member_number"].astype(str) + "_" +
    df["Date"].dt.strftime("%Y-%m-%d")
)



In [ ]:
df.head()

,Member_number,Date,itemDescription,Transaction_ID
0,1808,2015-07-21,tropical fruit,1808_2015-07-21
1,2552,2015-01-05,whole milk,2552_2015-01-05
2,2300,2015-09-19,pip fruit,2300_2015-09-19
3,1187,2015-12-12,other vegetables,1187_2015-12-12
4,3037,2015-02-01,whole milk,3037_2015-02-01


CUSTOMER SEGMENTATION USING KMEANS

In [ ]:
# Aggregate item-level data to customer-level behavior
customer_features = df.groupby("Member_number").agg(
    Num_Transactions=("Transaction_ID", "nunique"),
    Total_Items=("itemDescription", "count")
)

customer_features.head()

,Num_Transactions,Total_Items
Member_number,,
1000,5,13
1001,5,12
1002,4,8
1003,4,8
1004,8,21


In [ ]:
# Feature Scaling
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
customer_features["Segment"] = kmeans.fit_predict(scaled_features)
customer_features


,Num_Transactions,Total_Items,Segment
Member_number,,,
1000,5,13,0
1001,5,12,0
1002,4,8,0
1003,4,8,0
1004,8,21,2
...,...,...,...
4996,3,10,0
4997,2,6,1
4998,1,2,1


In [ ]:
# Convert cluster centers back to original scale
cluster_centers = pd.DataFrame(
    scaler.inverse_transform(kmeans.cluster_centers_),
    columns=customer_features.columns[:2]
)

cluster_centers["Cluster"] = cluster_centers.index

cluster_centers

,Num_Transactions,Total_Items,Cluster
0,4.454201,11.487682,0
1,2.192666,5.423749,1
2,6.943049,18.864322,2


In [ ]:
# Cluster Interpretation & Segment Naming
cluster_centers["Segment_Name"] = "Regular Buyers"

# Cluster with highest transactions → Frequent Buyers
cluster_centers.loc[
    cluster_centers["Num_Transactions"].idxmax(),
    "Segment_Name"
] = "Frequent Buyers"

# Cluster with lowest transactions → Occasional Buyers
cluster_centers.loc[
    cluster_centers["Num_Transactions"].idxmin(),
    "Segment_Name"
] = "Occasional Buyers"


segment_map = cluster_centers.set_index("Cluster")["Segment_Name"].to_dict()
segment_map

customer_features["Segment_Name"] = customer_features["Segment"].map(segment_map)
customer_features


,Num_Transactions,Total_Items,Segment,Segment_Name
Member_number,,,,
1000,5,13,0,Regular Buyers
1001,5,12,0,Regular Buyers
1002,4,8,0,Regular Buyers
1003,4,8,0,Regular Buyers
1004,8,21,2,Frequent Buyers
...,...,...,...,...
4996,3,10,0,Regular Buyers
4997,2,6,1,Occasional Buyers
4998,1,2,1,Occasional Buyers


In [ ]:
# Export to CSV
customer_features.reset_index().to_csv("customer_segments.csv", index=False)

MARKET BASKET ANALYSIS

In [ ]:
# Market Basket Matrix Creation

basket = (
    df.groupby(["Transaction_ID", "itemDescription"])
    .size()
    .unstack(fill_value=0)
)

# Convert counts to binary (1 = item present, 0 = absent)
basket = (basket > 0).astype(int)
basket

itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Transaction_ID,,,,,,,,,,,,,,,,,,,,,
1000_2014-06-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1000_2015-03-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1000_2015-05-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_2015-07-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_2015-11-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999_2015-05-16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4999_2015-12-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5000_2014-03-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Apriori Algorithm
frequent_itemsets = apriori(
    basket,
    min_support=0.001,
    use_colnames=True
)
frequent_itemsets.head()


,support,itemsets
0,0.004010,(Instant food products)
1,0.021386,(UHT-milk)
2,0.001470,(abrasive cleaner)
3,0.001938,(artif. sweetener)
4,0.008087,(baking powder)


In [ ]:
#Association Rule Generation
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.2
)

rules = rules[rules["lift"] >= 1.05]
rules = rules[
    ["antecedents", "consequents", "support", "confidence", "lift"]
]

rules.sort_values("lift", ascending=False).head()
global_rules = rules


SEGMENT-WISE MARKET BASKET ANALYSIS

In [ ]:
# Merge Segment Information Back To Transaction
df_seg = df.merge(
    customer_features[["Segment_Name"]],
    left_on="Member_number",
    right_index=True
)
df_seg.head()


,Member_number,Date,itemDescription,Transaction_ID,Segment_Name
0,1808,2015-07-21,tropical fruit,1808_2015-07-21,Regular Buyers
1,2552,2015-01-05,whole milk,2552_2015-01-05,Regular Buyers
2,2300,2015-09-19,pip fruit,2300_2015-09-19,Regular Buyers
3,1187,2015-12-12,other vegetables,1187_2015-12-12,Occasional Buyers
4,3037,2015-02-01,whole milk,3037_2015-02-01,Occasional Buyers


In [ ]:
#Function to perform Market Basket Analysis for a specific customer segment
def mba_by_segment(segment_name):
    temp = df_seg[df_seg["Segment_Name"] == segment_name]

    if temp.empty:
        return pd.DataFrame()

    basket = (
        temp.groupby(["Transaction_ID", "itemDescription"])
        .size()
        .unstack(fill_value=0)
    )

    basket = (basket > 0).astype(int)

    freq = apriori(basket, min_support=0.002, use_colnames=True)

    rules = association_rules(
        freq,
        metric="confidence",
        min_threshold=0.15
    )

    return rules[rules["lift"] >= 1.01]



In [ ]:
for seg, rules_df in segment_rules.items():
    if not rules_df.empty:
        print(f"\nSegment: {seg}")
        display(
            rules_df[
                ["antecedents", "consequents", "support", "confidence", "lift"]
            ]
            .sort_values("lift", ascending=False)
            .head()
        )



Segment: Regular Buyers


,antecedents,consequents,support,confidence,lift
3,(frozen meals),(other vegetables),0.002836,0.161290,1.334810
5,(grapes),(whole milk),0.002695,0.193878,1.218387
1,(cat food),(whole milk),0.002553,0.193548,1.216319
0,(candy),(whole milk),0.002978,0.185841,1.167881
11,(specialty bar),(whole milk),0.002269,0.175824,1.104934



Segment: Occasional Buyers


,antecedents,consequents,support,confidence,lift
10,(processed cheese),(root vegetables),0.002124,0.216216,3.271030
3,(frozen vegetables),(root vegetables),0.003716,0.155556,2.353324
4,(fruit/vegetable juice),(rolls/buns),0.005840,0.186441,1.696430
2,(candy),(rolls/buns),0.002124,0.181818,1.654370
5,(grapes),(other vegetables),0.002655,0.181818,1.556612



Segment: Frequent Buyers


,antecedents,consequents,support,confidence,lift
20,(processed cheese),(rolls/buns),0.002654,0.261905,2.375482
6,(condensed milk),(rolls/buns),0.002171,0.209302,1.898377
4,(chocolate),(rolls/buns),0.005066,0.185841,1.685579
9,(frankfurter),(rolls/buns),0.006273,0.159509,1.446752
16,(long life bakery product),(whole milk),0.004584,0.228916,1.349723


In [ ]:
#Best-Selling Product Analysis (Global)
global_bestsellers = (
    df["itemDescription"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "Recommended Product", "itemDescription": "Count"})
)


In [ ]:
global_bestsellers

,Count,count
0,whole milk,2502
1,other vegetables,1898
2,rolls/buns,1716
3,soda,1514
4,yogurt,1334
...,...,...
162,rubbing alcohol,5
163,bags,4
164,baby cosmetics,3
165,kitchen utensil,1


In [ ]:
# Export to CSV
global_bestsellers.to_csv("global_bestsellers.csv", index=False)


In [ ]:
#Segment-Wise Best-Selling Products
segment_bestsellers = (
    df_seg.groupby(["Segment_Name", "itemDescription"])
    .size()
    .reset_index(name="Count")
    .sort_values(["Segment_Name", "Count"], ascending=[True, False])
)


In [ ]:
segment_bestsellers

,Segment_Name,itemDescription,Count
161,Frequent Buyers,whole milk,749
101,Frequent Buyers,other vegetables,557
120,Frequent Buyers,rolls/buns,475
136,Frequent Buyers,soda,428
162,Frequent Buyers,yogurt,380
...,...,...,...
479,Regular Buyers,toilet cleaner,4
364,Regular Buyers,cream,3
392,Regular Buyers,hair spray,3
331,Regular Buyers,bags,2


In [ ]:
# Export to CSV
segment_bestsellers.to_csv("segment_bestsellers.csv", index=False)


In [ ]:
#global cross sell Recommendation
def global_cross_sell_recommendations(
    rules,
    min_confidence=0.05,
    min_lift=1.05,
    min_support=0.0003
):
    if rules.empty:
        return pd.DataFrame()

    recs = rules[
        (rules["confidence"] >= min_confidence) &
        (rules["lift"] >= min_lift) &
        (rules["support"] >= min_support)
    ].copy()

    recs["Trigger Product"] = recs["antecedents"].apply(
        lambda x: ", ".join(list(x))
    )
    recs["Recommended Product"] = recs["consequents"].apply(
        lambda x: list(x)[0]
    )

    return recs[
        ["Trigger Product", "Recommended Product"]
    ].drop_duplicates().reset_index(drop=True)



In [ ]:
global_cross_sell = global_cross_sell_recommendations(global_rules)
global_cross_sell.head()



,Trigger Product,Recommended Product
0,"sausage, rolls/buns",whole milk
1,"yogurt, sausage",whole milk


In [ ]:
global_cross_sell = global_cross_sell_recommendations(global_rules)

# Export to CSV
global_cross_sell.to_csv("global_cross_sell.csv", index=False)


In [ ]:
#Segment-Specific Cross-Sell Recommendation
def cross_sell_recommendations(
    rules,
    min_confidence=0.1,
    min_lift=1.2,
    min_support=0.001
):
    if rules.empty:
        return pd.DataFrame()

    recs = rules[
        (rules["confidence"] >= min_confidence)
        &(rules["lift"] >= min_lift)
        &(rules["support"] >= min_support)
        &(rules["antecedents"].apply(lambda x: len(x) == 1))
        &(rules["consequents"].apply(lambda x: len(x) == 1))
    ].copy()

    recs["Trigger Product"] = recs["antecedents"].apply(lambda x: list(x)[0])
    recs["Recommended Product"] = recs["consequents"].apply(lambda x: list(x)[0])


    return recs[
        ["Trigger Product", "Recommended Product"]
    ].drop_duplicates().reset_index(drop=True)


In [ ]:
cross_sell_by_segment = {
    seg: cross_sell_recommendations(rules)
    for seg, rules in segment_rules.items()
}


In [ ]:
#segment cross sell
all_cross_sell = []

for seg, df_cs in cross_sell_by_segment.items():
    if not df_cs.empty:
        df_cs["Segment"] = seg
        all_cross_sell.append(df_cs)

segment_cross_sell = pd.concat(all_cross_sell)

In [ ]:
segment_cross_sell.head()

,Trigger Product,Recommended Product,Segment
0,cat food,whole milk,Regular Buyers
1,frozen meals,other vegetables,Regular Buyers
2,grapes,whole milk,Regular Buyers
0,candy,rolls/buns,Occasional Buyers
1,frozen vegetables,root vegetables,Occasional Buyers


In [ ]:
# Export to CSV
pd.concat(all_cross_sell).to_csv("segment_cross_sell.csv", index=False)